# Data Normalization Mini Class

In [1]:
import os

import pandas as pd
import sqlite3

from data_norm_class.constants import (
    FILE_DATA_RAW,
    FOLDER_DATA_NORMALIZED
)
from data_norm_class.io import (
    read_raw_aarc_data
)

In [7]:
FOLDER_DATA_NORMALIZED = os.path.join("..", FOLDER_DATA_NORMALIZED)
FILE_DATA_RAW = os.path.join("..", FILE_DATA_RAW)

## What's the problem with single CSV files?

### Redundancy

### Integrity

### Scaling

## Data normalization

### Normal Forms

#### 1. Normal Form (1NF)

#### 2. Normal Form (2NF)

#### 3. Normal Form (3NF)

### Entity-Relationship diagrams

## Working with normalized data

### Transforming AARC to 3NF

### [Optional] Collections 

### Simple statistics

### Joining

## Advanced topics

### Missing data

### Delete cascades

### Databases